### Principal Component Analysis

## Reviews Preparation for Natural Language Processing

Add review_scores_rating from listings data to reviews data. Listings data only has review scores pertaining to the most recent review for a particular listing. This means that there will be many reviews that do not have a score, which we will remove during the merge.

In [38]:
#Read in libraries
import pandas as pd
import swifter

import numpy as np

import matplotlib.pyplot as plt
import seaborn as sns

import nltk

In [39]:
#Suppress future warnings
import warnings
warnings.simplefilter(action='ignore', category=FutureWarning)

In [40]:
#Increase number of columns and rows displayed by Pandas
pd.set_option('display.max_columns', 500)
pd.set_option('display.max_rows',100)

#Ignore warnings
import warnings; warnings.simplefilter('ignore')

In [41]:
#Set path to listings and review data
path = r'C:\Users\kishe\Documents\Data Science\Projects\Python Projects\In Progress\Air BnB - SF\Data\02_Intermediate/'

In [ ]:
#Parse in listings dates
date = ['calendar_last_scraped', 'calendar_updated', 'first_review' ,'host_since', 'last_review']

#Read in Airbnb Listings Data
listings = pd.read_csv(path + '01_04_2020_Listings_Cleaned.csv',parse_dates=date, index_col=0, low_memory=True, sep=',')

#Read in Airbnb Calendar and Reviews data
reviews = pd.read_csv(path + '01_04_2020_Reviews_Cleaned.csv', sep = ',',
                       parse_dates=['date'], low_memory=True,index_col=0)

In [ ]:
#Preview listings
listings.head().T

In [ ]:
#Preview reviews data
reviews.head()

**Merge review_scores_rating from listings to corresponding reviews**

Ratings on airbnb
At airbnb hosts and guests are not reviewed in same. Where guests simply get a written review hosts also receives a star rating from 1 to 5 on 6 parameters:

Accuracy
Communication
Cleanliness
Location
Check In
Value
which are also calculated into one overall rating.

In [ ]:
listings_cols=['host_is_superhost', 'host_response_time', 'latitude', 'longitude', 
               'neighbourhood_cleansed', 'number_of_reviews', 'room_type','id']

#Merge Review scores from listings to reviews dataframe. Merge on last review to confirm scores are assigned to proper review
reviews = reviews.merge(listings[listings_cols], left_on= ['listing_id'], 
                              right_on=['id'], suffixes=('_review', '_listings'))

#Drop duplicate values
reviews.drop_duplicates(inplace=True)

#Drop unnecessary columns from review_scores
reviews.drop(columns=['id_listings'], axis = 1, inplace= True)

#View review_scores shape
print('Data shape:', reviews.shape)

#Check
reviews.head().T

### Quick clean up for NLP

In [ ]:
#View missing values in review_scores
print('\nMissing values:\n', reviews.isna().sum())

In [ ]:
#Remove rows with host_response_time
reviews = reviews[-reviews.host_response_time.isna()]

#View updated reviews shape
print('Updated reviews data shape:',reviews.shape)

Sample Data 

We'll take a 5% sample for our analysis

In [ ]:
#Sample
sample = reviews.sample(frac=0.05,random_state=1)

#Sample Preview
print('Sample shape:', sample.shape)
display(sample.head().T)

Stop Word Removal

Language Parsing

In [ ]:
#Import stopwords
from nltk.corpus import stopwords

#check stopwords
stop =stopwords.words('english')
print(stop)

In [ ]:
#Exclude stopwords from comments
sample['comments_parsed'] = sample['comments'].apply(lambda x: ' '.join([word for word in x.split() if word not in (stop)]))

#Check
sample.head()

## Sentiment Analysis

In [ ]:
#Import and instantiate sentiment intensity analyzer
from vaderSentiment.vaderSentiment import SentimentIntensityAnalyzer
analyzer = SentimentIntensityAnalyzer()

#Write fuctions to capture positive, negative, neutral, and compound scores to later apply to reviews.comments_parsed
def neg_scores(comment):
    #Function to capture neg semantic score 
    score = analyzer.polarity_scores(comment)['neg']
    return score

def pos_scores(comment):
    #Function to capture positive semantic score 
    score = analyzer.polarity_scores(comment)['pos']
    return score

def neutral_scores(comment):
    #Function to capture negative semantic score 
    score = analyzer.polarity_scores(comment)['neu']
    return score

def compound_scores(comment):
    #Function to capture compound semantic score 
    score = analyzer.polarity_scores(comment)['compound']
    return score

In [ ]:
#Apply functions to reviews and assign scores to unique column
sample['sentiment_neg']= sample['comments_parsed'].swifter.apply(neg_scores)
sample['sentiment_pos']= sample['comments_parsed'].swifter.apply(pos_scores)
sample['sentiment_neu']= sample['comments_parsed'].swifter.apply(neutral_scores)
sample['sentiment_compound']= sample['comments_parsed'].swifter.apply(compound_scores)

In [ ]:
#Set path to write processed data
path = r'C:\Users\kishe\Documents\Data Science\Projects\Python Projects\In Progress\Air BnB - SF\Data\03_Processed'

#Write to csv
sample.to_csv(path + '/01_10_2020_Reviews_Processed_Text_Analysis.csv',sep=',', index=False)